In [1]:
## Michael Gallotta
## dunkins1.ipynb
## 2019/05/05
##
## scrapes data from menuism.com of locations of dunkin (donuts) in the state of MA
##
## output pandas df contains zipcodes 
## and street addresses within each zipcode. 
##

## 20190422 random thought: what if i were to host this data on agol. 
## other web based publishing solutions? runs dunkins.ipynb, updates dataset, draws map. 
## what about copyrights? 

## 20190505 look into farthest euclidian disance from all dd locations 

## 20190721 figured it out. turns out I was overthinking it. can you believe that? 

import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
# requests data from the menuism website

menuism = "https://www.menuism.com/restaurant-locations/dunkin-donuts-181624/us/ma"
MA_WIKI = "https://en.wikipedia.org/wiki/List_of_municipalities_in_Massachusetts"

page = requests.get(menuism)
html = page.content

soup = bs(html, "html.parser")
parse_soup = soup.find_all("ul", {"class":"list-unstyled-links"})

dd_locs = parse_soup[0].find_all("li",recursive=True)

In [3]:
def jump_link(url):    
    jump_page = requests.get(url)
    jump_html = jump_page.content

    jump_soup = bs(jump_html, "html.parser")
    jump_parse_soup = jump_soup.find_all("div", {"class":"map-info"})

    jump_dd_locs = jump_parse_soup[0].find_all("span",recursive=True)

    jump_address_ls = []

    jump_street = jump_dd_locs[0].get_text(strip = True)
    jump_town = jump_dd_locs[1].get_text(strip = True)
    jump_zip = jump_dd_locs[3].get_text(strip = True)

    jump_address_ls.append(jump_street)
    jump_address_ls.append(jump_town)
    jump_address_ls.append("Massachusetts")
    jump_address_ls.append(jump_zip[0:5])
    
    return(jump_address_ls)

In [4]:
complete_address_list = []


for tags in dd_locs:
    if tags.h5:
        town_name = tags.h5['id']
    elif tags.em:
        address_ls = []
        address = tags.em
        zip_code = tags.text[67:72]
        address_ls.append(address.text)
        address_ls.append(town_name.title())
        address_ls.append("Massachusetts")
        address_ls.append(zip_code[0:5])
        complete_address_list.append(address_ls)
    else:
        menuism_jump_url = tags.a['href']
        
        complete_address_list.append(jump_link(menuism_jump_url))

In [5]:
print("This list has %i Dunkin' Donuts Locations" %len(complete_address_list))

complete_address_list

This list has 1101 Dunkin' Donuts Locations


[['147 North Quincy Street', 'Abington', 'Massachusetts', '02351'],
 ['336 Centre Ave', 'Abington', 'Massachusetts', '02351'],
 ['375 Centre Ave', 'Abington', 'Massachusetts', '02351'],
 ['937 Bedford St', 'Abington', 'Massachusetts', '02351'],
 ['259 Brockton Ave', 'Abington', 'Massachusetts', '02351'],
 ['212 Main St', 'Acton', 'Massachusetts', '01720'],
 ['299 Main St', 'Acton', 'Massachusetts', '01720'],
 ['44 Great Rd', 'Acton', 'Massachusetts', '01720'],
 ['182 Great Rd', 'Acton', 'Massachusetts', '01720'],
 ['100 Powdermill Rd', 'Acton', 'Massachusetts', '01720'],
 ['150 S Main St', 'Acushnet', 'Massachusetts', '02743'],
 ['177 Columbia St', 'Adams', 'Massachusetts', '01220'],
 ['12 Springfield St', 'Agawam', 'Massachusetts', '01001'],
 ['152 Silver St', 'Agawam', 'Massachusetts', '01001'],
 ['509 Cambridge St', 'Allston', 'Massachusetts', '02134'],
 ['209 N Harvard St', 'Allston', 'Massachusetts', '02134'],
 ['179 Brighton Ave Ste F', 'Allston', 'Massachusetts', '02134'],
 ['21